<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [1]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [2]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [3]:
display(apps)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [4]:
apps = apps.drop(columns=['Type', 'Price', 'Category','Last Updated', 'Current Ver'])
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19M,"10,000+",Everyone,Art & Design,4.0.3 and up
1,Coloring book moana,3.9,967,14M,"500,000+",Everyone,Art & Design;Pretend Play,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8.7M,"5,000,000+",Everyone,Art & Design,4.0.3 and up
3,Sketch - Draw & Paint,4.5,215644,25M,"50,000,000+",Teen,Art & Design,4.2 and up
4,Pixel Draw - Number Art Coloring Book,4.3,967,2.8M,"100,000+",Everyone,Art & Design;Creativity,4.4 and up


In [5]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','a', regex=True).str.replace('a','.', 1, regex=True).str.replace('a','', regex=True)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})

C:\Users\rafal\AppData\Local\Temp\ipykernel_20328\2251694235.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\rafal\AppData\Local\Temp\ipykernel_20328\2251694235.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\rafal\AppData\Local\Temp\ipykernel_20328\2251694235.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [6]:
class DataClean:
    @staticmethod
    def splitGenres(x):
        genresConnected = x["Genres"].unique().tolist()
        genres = set(())
        for i in genresConnected:
            afterSplit = i.split(";")
            for j in afterSplit:
                if j!='February 11, 2018':
                    genres.add(j)
        for i in list(genres):
            x[i] = [0 for x in range(x.shape[0])]
        for index in x.index:
            values = x.loc[index,'Genres'].split(";")
            for i in values:
                x.loc[index,i] = 1
        return x

In [7]:
apps = DataClean.splitGenres(apps)
apps.head()
apps.drop(columns=['Genres'])

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Video Players & Editors,Dating,Adventure,...,Comics,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159.0,19.0,10000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,3.9,967.0,14.0,500000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510.0,8.7,5000000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,4.5,215644.0,25.0,50000000.0,Teen,4.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,4.3,967.0,2.8,100000.0,Everyone,4.40,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10832,FR Tides,3.8,1195.0,582.0,100000.0,Everyone,2.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10833,Chemin (fr),4.8,44.0,619.0,1000.0,Everyone,2.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10834,FR Calculator,4.0,7.0,2.6,500.0,Everyone,4.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10836,Sya9a Maroc - FR,4.5,38.0,53.0,5000.0,Everyone,4.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [9]:
DataProcessing.shuffle(apps)
apps = apps.reset_index(drop=True)

In [10]:
DataProcessing.normalization(apps)

In [11]:
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Comics,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word
0,Microsoft PowerPoint,0.875,0.013784,0.036395,0.10000,Everyone,Productivity,0.485714,0,0,...,0,0,1,0,0,0,0,0,0,0
1,YouTube Kids,0.875,0.010471,0.036395,0.05000,Everyone,Entertainment;Music & Video,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
2,WFLA News Channel 8 - Tampa FL,0.700,0.000003,0.013092,0.00001,Everyone,News & Magazines,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,DJ Mix Effects Simulator,0.775,0.000134,0.019134,0.00100,Everyone,Simulation,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Middlesbrough FC Official,0.875,0.000003,0.035247,0.00001,Everyone,Sports,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.999)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Comics,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word
0,Smart Air Conditioner,0.575,0.000107,0.012085,0.00050,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pyaterochka,0.525,0.000187,0.036395,0.00100,Everyone,Food & Drink,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Service CU Mobile Banking,0.925,0.000138,0.013092,0.00010,Everyone,Finance,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Gang Sniper V. Pocket Edition.,0.725,0.000136,0.060423,0.00050,Mature 17+,Action,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Little Teddy Bear Colouring Book Game,0.800,0.000002,0.007049,0.00010,Everyone,Art & Design,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
5,DW Spectrum™ IP VMS,0.600,0.000002,0.001410,0.00001,Everyone,Business,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0
6,ao,0.625,0.000315,0.004935,0.00100,Everyone,Strategy,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,0
7,FREE Stock Market Trading Tips,0.725,0.000016,0.002618,0.00005,Everyone,Finance,0.190000,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
apps["Content Rating"].unique().tolist()

['Everyone',
 'Teen',
 'Mature 17+',
 'Everyone 10+',
 'Unrated',
 'Adults only 18+']

In [14]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [15]:
validatingSet.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Comics,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word
0,Smart Air Conditioner,0.575,0.000107,0.012085,0.0005,Adults only 18+,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pyaterochka,0.525,0.000187,0.036395,0.0010,Adults only 18+,Food & Drink,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Service CU Mobile Banking,0.925,0.000138,0.013092,0.0001,Adults only 18+,Finance,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Gang Sniper V. Pocket Edition.,0.725,0.000136,0.060423,0.0005,Adults only 18+,Action,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Little Teddy Bear Colouring Book Game,0.800,0.000002,0.007049,0.0001,Adults only 18+,Art & Design,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2>k-nearest neighbors algorithm</h2>

In [16]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return sum**(1/2)
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNN.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h2>Apps Recommendation</h2>

<h4>Method 1</h4>

In [17]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNN.clustering(trainingSet.copy(), validatingSet.iloc[0])
recommendedApps.sort_values("distance").head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word,distance
6227,SportCAM,0.55,9.489042e-06,0.008560,0.00010,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.025251
7013,DW VMAX,0.60,1.875534e-05,0.007553,0.00010,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.025411
1750,Smart Air Conditioner(CAC),0.55,9.088097e-06,0.006949,0.00005,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.025526
1875,AZ PLAYER HD,0.55,5.123192e-07,0.003726,0.00001,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.026365
2900,U-Scan,0.60,1.744113e-05,0.041289,0.00005,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.038446


In [18]:
validatingSet.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Comics,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word
0,Smart Air Conditioner,0.575,0.000107,0.012085,0.0005,Adults only 18+,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pyaterochka,0.525,0.000187,0.036395,0.0010,Adults only 18+,Food & Drink,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Service CU Mobile Banking,0.925,0.000138,0.013092,0.0001,Adults only 18+,Finance,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Gang Sniper V. Pocket Edition.,0.725,0.000136,0.060423,0.0005,Adults only 18+,Action,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Little Teddy Bear Colouring Book Game,0.800,0.000002,0.007049,0.0001,Adults only 18+,Art & Design,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>Method 2</h4>

In [19]:
KNN.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word,distance
6732,Synd e-Passbook,0.825,2.093604e-04,0.005136,9.999990e-04,Everyone,Finance,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.538264
563,NSE Mobile Trading,0.775,3.088839e-04,0.000403,9.999990e-04,Everyone,Finance,0.157143,0,0,...,0,0,0,0,0,0,0,0,0,0.538349
6312,Financial Calculator Pro EF,0.825,1.409992e-05,0.001007,4.999900e-05,Everyone,Finance,0.157143,0,0,...,0,0,0,0,0,0,0,0,0,0.538396
3178,US Stock Market,0.850,4.278979e-05,0.003223,9.999900e-05,Everyone,Finance,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.538583
2653,Banking Calculator,0.875,8.248339e-05,0.003625,9.999900e-05,Everyone,Finance,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.539032
3461,CB Bank Mobile Banking,0.875,2.911310e-05,0.002719,9.999900e-05,Everyone,Finance,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.539091
7316,Rewards,0.700,1.247386e-06,0.003021,4.999000e-06,Everyone,Finance,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.555276
7959,Southland CU,0.700,2.405673e-06,0.001813,9.999000e-06,Everyone,Finance,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.555315
6822,Space Coast CU Mobile,0.700,2.051504e-05,0.001108,9.999900e-05,Everyone,Finance,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.555347
993,dt Pro,0.950,6.682424e-08,0.003525,9.990000e-07,Everyone,Finance,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.557974


<h2>Importing Reviews Database</h2>

In [20]:
reviews = pd.read_csv(r"processedReviews.csv")

In [21]:
display(reviews)

,App,reviews
0,10 Best Foods for You,best;food;health;healthy;love
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,easy;job;use;useful;mobile
2,11st,and;not;you;for;english
3,1800 Contacts - Lens Store,easy;contacts;order;super;love
4,1LINE – One Line with One Touch,game;play;love;ads;many
...,...,...
860,Hotels.com: Book Hotel Rooms & Find Vacation D...,hotel;book;pet;friendly;find
861,Hotspot Shield Free VPN Proxy & Wi-Fi Security,trial;days;free;asking;credit
862,Hotstar,shows;answer;time;u;playing
863,Hotwire Hotel & Car Rental App,would;hotels;hotel;better;card


In [22]:
appsWithReviews = pd.merge(apps, reviews, on="App")

In [23]:
display(appsWithReviews)

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word,reviews
0,Freeme Launcher—Stylish Theme,0.775,0.000063,0.006445,0.00010,Everyone,Personalization,0.442857,0,0,...,1,0,0,0,0,0,0,0,0,launcher;themes;drawer;option;awesome
1,ColorSnap® Visualizer,0.600,0.000073,0.076536,0.00100,Everyone,House & Home,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,color;paint;work;even;phone
2,DEER HUNTER 2018,0.825,0.021287,0.081571,0.01000,Teen,Action,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,slow;uninstall;reinstall;hunt;movement
3,DEER HUNTER 2018,0.825,0.021286,0.081571,0.01000,Teen,Action,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,slow;uninstall;reinstall;hunt;movement
4,Dresses Ideas & Fashions +3000,0.875,0.000011,0.007251,0.00010,Mature 17+,Beauty,0.085714,0,0,...,0,0,0,0,0,0,0,0,0,dress;see;and;love;ive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,Fortune City - A Finance App,0.900,0.001098,0.090634,0.00050,Everyone,Finance,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,track;fun;way;keep;spending
900,Bualuang mBanking,0.750,0.001079,0.009063,0.00500,Everyone,Finance,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,not;of;have;bank;what
901,GRE Tutor,0.750,0.000006,0.001309,0.00005,Everyone,Education,0.157143,0,0,...,0,0,0,0,0,0,0,0,0,words;word;would;gre;better
902,DIY Garden Ideas,0.775,0.000074,0.012085,0.00050,Everyone,Lifestyle,0.428571,0,0,...,0,0,0,0,0,0,0,1,0,ideas;better;interesting;not;garden


In [24]:
trainingSet, validatingSet = DataProcessing.split(appsWithReviews, 0.995)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Personalization,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word,reviews
0,Bualuang mBanking,0.750,0.001079,0.009063,0.00500,Everyone,Finance,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,not;of;have;bank;what
1,GRE Tutor,0.750,0.000006,0.001309,0.00005,Everyone,Education,0.157143,0,0,...,0,0,0,0,0,0,0,0,0,words;word;would;gre;better
2,DIY Garden Ideas,0.775,0.000074,0.012085,0.00050,Everyone,Lifestyle,0.428571,0,0,...,0,0,0,0,0,0,0,1,0,ideas;better;interesting;not;garden
3,Cisco Webex Teams,0.800,0.000037,0.045317,0.00010,Everyone,Business,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,spark;message;call;phone;calls


In [25]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [26]:
class KNNWithJaccardSimilarityCoefficient:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)-1):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return (sum**(1/2)) * KNNWithJaccardSimilarityCoefficient.jaccard(v1[len(v1)-1], v2[len(v1)-1])
    
    def jaccard(string1,string2):
        sim = 0
        try:
            string1 = string1.split(";")
        except:
            string1 = [""]       
        try:
            string2 = string2.split(";")
        except:
            string2 = [""]
            
        for word in string1:
            if word in string2:
                sim+=1
        return 1 - (sim/5)
    
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNNWithJaccardSimilarityCoefficient.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNNWithJaccardSimilarityCoefficient.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h4>Method 1</h4>

In [29]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:-1]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
revs = []
for i in validatingSet["reviews"].tolist():
    i = i.split(";")
    for j in i:
        revs.append(j)
words = {}
for word in revs:
        words[word] = 0
for word in revs:
        words[word] +=1
revs = []
for i in range(5):
    try:
        revs.append(max(words, key=words.get))
        del words[max(words, key=words.get)]
    except:
        pass
avgWords = ";".join(revs)
avgAppList.append(avgWords)

avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNNWithJaccardSimilarityCoefficient.clustering(trainingSet.copy(), validatingSet.iloc[0])
recommendedApps.sort_values("distance").head(10)

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word,reviews,distance
888,Expense IQ Money Manager,0.850,0.000480,0.026183,0.0010,Everyone,Finance,0.428571,0,0,...,0,0,0,0,0,0,0,0,update;account;paid;using;years,0.101535
706,Fresh EBT - Food Stamp Balance,0.875,0.000443,0.006445,0.0010,Everyone,Finance,0.432857,0,0,...,0,0,0,0,0,0,0,0,balance;card;would;love;coupons,0.125093
212,Branch,0.900,0.001559,0.002820,0.0010,Everyone,Finance,0.428571,0,0,...,0,0,0,0,0,0,0,0,loan;branch;best;time;phone,0.150184
558,11st,0.700,0.001085,0.019134,0.0100,Everyone,Shopping,0.428571,0,0,...,0,0,0,0,0,0,0,0,and;not;you;for;english,1.132113
399,H Pack,0.825,0.000210,0.000705,0.0010,Everyone,Lifestyle,0.432857,0,0,...,0,0,0,0,0,0,1,0,not;use;will;true;problems,1.132985
843,Caller ID +,0.750,0.000212,0.117825,0.0010,Everyone,Communication,0.185714,0,0,...,0,0,0,0,0,0,0,0,found;what;scam;havent;even,1.134716
60,Candy Crush Jelly Saga,0.825,0.028968,0.077543,0.0500,Everyone,Puzzle,0.428571,0,0,...,0,0,0,0,0,0,0,0,game;get;not;level;facebook,1.135075
61,Candy Crush Jelly Saga,0.825,0.028971,0.077543,0.0500,Everyone,Puzzle,0.428571,0,0,...,0,0,0,0,0,0,0,0,game;get;not;level;facebook,1.135075
656,"FUN Keyboard – Emoji Keyboard, Sticker,Theme &...",0.925,0.000269,0.026183,0.0005,Everyone,Personalization,0.428571,0,0,...,0,0,0,0,0,0,0,0,fun;thank;love;add;not,1.140088
879,Entel,0.550,0.000360,0.054381,0.0010,Everyone,Lifestyle,0.432857,0,0,...,0,0,0,0,0,0,1,0,of;can;my;time;application,1.143208


<h4>Method 2</h4>

In [28]:
KNNWithJaccardSimilarityCoefficient.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Video Players & Editors,Dating,...,Productivity,Card,Role Playing,Medical,Casual,Racing,Lifestyle,Word,reviews,distance
212,Branch,0.900,0.001559,0.002820,0.00100,Everyone,Finance,0.428571,0,0,...,0,0,0,0,0,0,0,0,loan;branch;best;time;phone,0.901549
540,AdWords Express,0.775,0.000159,0.010070,0.00100,Everyone,Business,0.428571,0,0,...,0,0,0,0,0,0,0,0,you;working;google;business;time,0.957293
541,AdWords Express,0.775,0.000159,0.010070,0.00100,Everyone,Business,0.428571,0,0,...,0,0,0,0,0,0,0,0,you;working;google;business;time,0.957293
71,Home Scouting® MLS Mobile,0.825,0.000034,0.006747,0.00010,Everyone,Lifestyle,0.171429,0,0,...,0,0,0,0,0,0,1,0,code;listings;waste;see;homes,0.960486
772,Easy Hair Style Design,0.825,0.000013,0.004129,0.00010,Everyone,Lifestyle,0.185714,0,0,...,0,0,0,0,0,0,1,0,love;hair;amazing;think;try,0.960628
750,Easy Makeup Tutorials,0.825,0.000162,0.004028,0.00100,Everyone,Lifestyle,0.185714,0,0,...,0,0,0,0,0,0,1,0,makeup;pictures;scam;love;learn,0.960634
636,Brit + Co,0.725,0.000022,0.003525,0.00001,Everyone,Lifestyle,0.428571,0,0,...,0,0,0,0,0,0,1,0,load;articles;frustrating;article;would,0.960667
635,Brit + Co,0.725,0.000022,0.003525,0.00001,Everyone,Lifestyle,0.428571,0,0,...,0,0,0,0,0,0,1,0,load;articles;frustrating;article;would,0.960667
12,Beautiful Design Birthday Cake,0.825,0.000015,0.002014,0.00050,Everyone,Lifestyle,0.185714,0,0,...,0,0,0,0,0,0,1,0,game;cake;think;beautiful;cakes,0.960776
305,Call Blocker,0.900,0.004206,0.002216,0.00500,Everyone,Business,0.428571,0,0,...,0,0,0,0,0,0,0,0,block;annoying;finally;calls;far,0.976402
